## Data Mining Project - Olist Sentiment Analysis

### Loading Libraries and Datasets

In [86]:
# %pip install tensorflow
# %pip install -U scikit-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [125]:
data_cust = pd.read_csv("datasets/olist_customers_dataset.csv")
data_order_item = pd.read_csv("datasets/olist_order_items_dataset.csv")
data_order_paym = pd.read_csv("datasets/olist_order_payments_dataset.csv")
data_order_review = pd.read_csv("datasets/olist_order_reviews_dataset.csv")
data_order_main = pd.read_csv("datasets/olist_orders_dataset.csv")
data_product = pd.read_csv("datasets/olist_products_dataset.csv")
data_sellers = pd.read_csv("datasets/olist_sellers_dataset.csv")
data_product_cate_name = pd.read_csv("datasets/product_category_name_translation.csv")

File Merger

In [129]:
compiled_df = data_cust.merge(data_order_main, how='inner', on='customer_id')
compiled_df = compiled_df.merge(data_order_review, how='inner', on='order_id')
compiled_df = compiled_df.merge(data_order_paym, how='inner', on='order_id')
compiled_df = compiled_df.merge(data_order_item, how='inner', on='order_id')
compiled_df = compiled_df.merge(data_product, how='inner', on='product_id')
compiled_df = compiled_df.merge(data_sellers, how='inner', on='seller_id')
compiled_df = compiled_df.merge(data_product_cate_name, how='inner', on='product_category_name')

In [130]:
compiled_df.dropna(subset=['order_delivered_carrier_date', 'order_delivered_customer_date','product_weight_g', 'product_length_cm', 'product_width_cm', 'product_height_cm']).reset_index()
compiled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115609 entries, 0 to 115608
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    115609 non-null  object 
 1   customer_unique_id             115609 non-null  object 
 2   customer_zip_code_prefix       115609 non-null  int64  
 3   customer_city                  115609 non-null  object 
 4   customer_state                 115609 non-null  object 
 5   order_id                       115609 non-null  object 
 6   order_status                   115609 non-null  object 
 7   order_purchase_timestamp       115609 non-null  object 
 8   order_approved_at              115595 non-null  object 
 9   order_delivered_carrier_date   114414 non-null  object 
 10  order_delivered_customer_date  113209 non-null  object 
 11  order_estimated_delivery_date  115609 non-null  object 
 12  review_id                     

Feature Selection

In [204]:
# converting to datetime
compiled_df['order_purchase_timestamp'] = pd.to_datetime(compiled_df.order_purchase_timestamp)
compiled_df['order_approved_at'] = pd.to_datetime(compiled_df.order_approved_at)
compiled_df['order_estimated_delivery_date'] = pd.to_datetime(compiled_df.order_estimated_delivery_date)
compiled_df['order_delivered_customer_date'] = pd.to_datetime(compiled_df.order_delivered_customer_date)
compiled_df.info()

# Drop columns
compiled_df1 = compiled_df[['order_status','freight_value',
                 'order_purchase_timestamp', 'order_approved_at', 'order_estimated_delivery_date', 
                 'order_delivered_customer_date', 'customer_state', 
                 'product_category_name_english', 'product_name_lenght', 'product_description_lenght', 
                 'product_photos_qty', 'review_score']]
compiled_df1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 115609 entries, 0 to 115608
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   customer_id                    115609 non-null  object        
 1   customer_unique_id             115609 non-null  object        
 2   customer_zip_code_prefix       115609 non-null  int64         
 3   customer_city                  115609 non-null  object        
 4   customer_state                 115609 non-null  object        
 5   order_id                       115609 non-null  object        
 6   order_status                   115609 non-null  object        
 7   order_purchase_timestamp       115609 non-null  datetime64[ns]
 8   order_approved_at              115595 non-null  datetime64[ns]
 9   order_delivered_carrier_date   114414 non-null  object        
 10  order_delivered_customer_date  113209 non-null  datetime64[ns]
 11  

Split of dataset

In [205]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(compiled_df1, compiled_df1['review_score']):
    strat_train_set = compiled_df1.loc[train_index]
    strat_test_set = compiled_df1.loc[test_index]

## Ordinary regression

## Feature Engineering
Aim: Predict rating score based on delivery performance

In [235]:
# %pip install statsmodels
# %pip install workalendar
from workalendar.america import Brazil
from sklearn.base import BaseEstimator, TransformerMixin

# Custom transformer to conduct feature engineering for both new/unseen data
class AttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        df = X.copy()
        
        # Calculate the estimated delivery time and actual delivery time in working days. 
        # This would allow us to exclude hollidays that could influence delivery times.
        # If the order_delivered_customer_date is null, it returns 0.
        df['wd_estimated_delivery_time'] = df['order_approved_at'] - df['order_estimated_delivery_date']
        df['wd_actual_delivery_time'] = df['order_approved_at'] - df['order_delivered_customer_date']
        
        # Calculate the time between the actual and estimated delivery date. If negative was delivered early, if positive was delivered late.
        df['wd_delivery_time_delta'] = df.wd_actual_delivery_time - df.wd_estimated_delivery_time

        # Calculate the time between the actual and estimated delivery date. If negative was delivered early, if positive was delivered late.
        df['is_late'] = df.order_delivered_customer_date > df.order_estimated_delivery_date
        
        # Calculate the order freight ratio.
        df['purchase_dayofweek'] = df.order_purchase_timestamp.dt.dayofweek
        
        # With that we can remove the timestamps from the dataset
        cols_drop = ['order_purchase_timestamp', 'order_approved_at', 'order_estimated_delivery_date', 'order_delivered_customer_date']
        df.drop(cols_drop, axis=1, inplace=True)
        
        return df


# Check if new features successfully created
attr_adder = AttributesAdder()
feat_eng = attr_adder.transform(strat_train_set)
feat_eng['review_score'] = feat_eng['review_score'].astype('category')
feat_eng

# # Seperate review_score from features 
# orders_features = feat_eng.drop('review_score', axis=1)
# orders_labels = feat_eng['review_score'].copy().dropna()
# orders_labels = orders_labels.astype('category')

,order_status,freight_value,customer_state,product_category_name_english,product_name_lenght,product_description_lenght,product_photos_qty,review_score,wd_estimated_delivery_time,wd_actual_delivery_time,wd_delivery_time_delta,is_late,purchase_dayofweek
88445,delivered,17.60,RJ,garden_tools,60.0,959.0,2.0,5,-24 days +17:35:15,-11 days +01:57:23,12 days 08:22:08,False,4
91154,delivered,19.35,SC,small_appliances,56.0,146.0,1.0,4,-29 days +21:58:47,-13 days +06:16:29,15 days 08:17:42,False,1
51418,delivered,17.06,BA,toys,45.0,293.0,2.0,5,-22 days +09:10:21,-12 days +11:23:52,10 days 02:13:31,False,6
33204,delivered,15.56,PR,housewares,57.0,698.0,3.0,1,-21 days +05:15:56,-20 days +19:58:01,1 days 14:42:05,False,3
135,delivered,66.98,GO,office_furniture,43.0,1128.0,1.0,5,-38 days +09:14:41,-21 days +13:50:00,17 days 04:35:19,False,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6934,delivered,17.81,RJ,furniture_decor,63.0,458.0,2.0,1,-25 days +02:07:56,-66 days +05:03:22,-41 days +02:55:26,True,5
4550,delivered,17.63,ES,furniture_decor,59.0,2432.0,2.0,5,-28 days +03:10:20,-7 days +14:46:38,21 days 11:36:18,False,2
56060,delivered,18.23,MG,computers_accessories,47.0,501.0,5.0,5,-26 days +10:34:35,-7 days +16:35:41,19 days 06:01:06,False,3
12651,delivered,14.10,DF,auto,57.0,804.0,7.0,3,-26 days +14:25:14,-12 days +23:12:47,14 days 08:47:33,False,3


In [244]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

# Ordered probit model
mod_log = OrderedModel(feat_eng['review_score'],
                        feat_eng[['is_late']],
                        distr='logit')

res_prob = mod_log.fit(method='bfgs')
res_prob.summary()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/statsmodels/miscmodels/ordinal_model.py:206: Warning: the endog has ordered == False, risk of capturing a wrong order for the categories. ordered == True preferred.
  warnings.warn("the endog has ordered == False, "


Optimization terminated successfully.
         Current function value: 1.187428
         Iterations: 19
         Function evaluations: 22
         Gradient evaluations: 22


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:           review_score   Log-Likelihood:            -1.0982e+05
Model:                   OrderedModel   AIC:                         2.197e+05
Method:            Maximum Likelihood   BIC:                         2.197e+05
Date:                Wed, 16 Nov 2022                                         
Time:                        14:13:34                                         
No. Observations:               92487                                         
Df Residuals:                   92482                                         
Df Model:                           5                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
is_late       -1.9594      0.024    -81.413      0.000      -2.007      -1.912
1/2           -2.1846      0.011   -201.105      0.000      -2.206      -2.163
2/3           -1.1701      0.017    -67.439      0.000      -1.204      -1.136
3/4           -0.5595      0.011    -50.932      0.000      -0.581      -0.538
4/5           -0.0877      0.007    -12.469      0.000      -0.101      -0.074
==============================================================================
"""